In [1]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


import numpy as np
import datetime
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display

import sklearn as skl
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier


In [2]:
# Loading data (from .csv)
# This needs to change to PostgreSQL query ('panda_query_from_postgres' file)
file_path = Path('Resources/master_opt_hist.csv')
df_optionsML = pd.read_csv(file_path)
df_optionsML.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,ticker,call_put,exp_date
0,AAPL220107C00105000,1/5/2022 19:28,105.0,72.25,66.8,67.25,0.00,0.000000,6.0,12.0,2.468754,True,REGULAR,USD,AAPL,C,1/7/2022
1,AAPL220107C00110000,1/5/2022 14:35,110.0,68.45,61.8,62.25,0.00,0.000000,8.0,4.0,2.250004,True,REGULAR,USD,AAPL,C,1/7/2022
2,AAPL220107C00115000,1/5/2022 20:18,115.0,61.25,56.8,57.25,0.00,0.000000,3.0,3.0,2.031255,True,REGULAR,USD,AAPL,C,1/7/2022
3,AAPL220107C00120000,1/6/2022 17:16,120.0,53.00,51.9,52.50,-5.25,-9.012876,8.0,412.0,2.398442,True,REGULAR,USD,AAPL,C,1/7/2022
4,AAPL220107C00125000,1/3/2022 20:31,125.0,57.11,46.9,47.55,0.00,0.000000,5.0,5.0,2.199223,True,REGULAR,USD,AAPL,C,1/7/2022


In [3]:
# Fill missing values with '0'
df_optionsML.fillna(value=0, inplace=True)

# Change 'inTheMoney' from boolean to float64
df_optionsML['inTheMoney'] = df_optionsML['inTheMoney'].astype('float')

In [4]:
# Change 'lastTradeDate' to datetime64
df_optionsML['lastTradeDate'] = pd.to_datetime(df_optionsML['lastTradeDate'])

In [5]:
# Change 'exp_date' to datetime64
df_optionsML['exp_date'] = pd.to_datetime(df_optionsML['exp_date'])

In [6]:
# Split 'lastTradeDate' into separate components, as int64
df_optionsML['lastTradeDate_month'] = df_optionsML['lastTradeDate'].dt.month
df_optionsML['lastTradeDate_day'] = df_optionsML['lastTradeDate'].dt.day
df_optionsML['lastTradeDate_year'] = df_optionsML['lastTradeDate'].dt.year
df_optionsML['lastTradeDate_hour'] = df_optionsML['lastTradeDate'].dt.hour
df_optionsML['lastTradeDate_min'] = df_optionsML['lastTradeDate'].dt.minute

# Split 'exp_date' into separate components, as int64
df_optionsML['exp_date_month'] = df_optionsML['exp_date'].dt.month
df_optionsML['exp_date_day'] = df_optionsML['exp_date'].dt.day
df_optionsML['exp_date_year'] = df_optionsML['exp_date'].dt.year
df_optionsML.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,call_put,exp_date,lastTradeDate_month,lastTradeDate_day,lastTradeDate_year,lastTradeDate_hour,lastTradeDate_min,exp_date_month,exp_date_day,exp_date_year
0,AAPL220107C00105000,2022-01-05 19:28:00,105.0,72.25,66.8,67.25,0.00,0.000000,6.0,12.0,...,C,2022-01-07,1,5,2022,19,28,1,7,2022
1,AAPL220107C00110000,2022-01-05 14:35:00,110.0,68.45,61.8,62.25,0.00,0.000000,8.0,4.0,...,C,2022-01-07,1,5,2022,14,35,1,7,2022
2,AAPL220107C00115000,2022-01-05 20:18:00,115.0,61.25,56.8,57.25,0.00,0.000000,3.0,3.0,...,C,2022-01-07,1,5,2022,20,18,1,7,2022
3,AAPL220107C00120000,2022-01-06 17:16:00,120.0,53.00,51.9,52.50,-5.25,-9.012876,8.0,412.0,...,C,2022-01-07,1,6,2022,17,16,1,7,2022
4,AAPL220107C00125000,2022-01-03 20:31:00,125.0,57.11,46.9,47.55,0.00,0.000000,5.0,5.0,...,C,2022-01-07,1,3,2022,20,31,1,7,2022


In [7]:
df_optionsML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23876 entries, 0 to 23875
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   contractSymbol       23876 non-null  object        
 1   lastTradeDate        23876 non-null  datetime64[ns]
 2   strike               23876 non-null  float64       
 3   lastPrice            23876 non-null  float64       
 4   bid                  23876 non-null  float64       
 5   ask                  23876 non-null  float64       
 6   change               23876 non-null  float64       
 7   percentChange        23876 non-null  float64       
 8   volume               23876 non-null  float64       
 9   openInterest         23876 non-null  float64       
 10  impliedVolatility    23876 non-null  float64       
 11  inTheMoney           23876 non-null  float64       
 12  contractSize         23876 non-null  object        
 13  currency             23876 non-

In [8]:
# Drop columns 
df_optionsML.drop(['contractSymbol','lastTradeDate','exp_date','contractSize','currency'], axis=1, inplace=True)
df_optionsML.head()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,ticker,call_put,lastTradeDate_month,lastTradeDate_day,lastTradeDate_year,lastTradeDate_hour,lastTradeDate_min,exp_date_month,exp_date_day,exp_date_year
0,105.0,72.25,66.8,67.25,0.00,0.000000,6.0,12.0,2.468754,1.0,AAPL,C,1,5,2022,19,28,1,7,2022
1,110.0,68.45,61.8,62.25,0.00,0.000000,8.0,4.0,2.250004,1.0,AAPL,C,1,5,2022,14,35,1,7,2022
2,115.0,61.25,56.8,57.25,0.00,0.000000,3.0,3.0,2.031255,1.0,AAPL,C,1,5,2022,20,18,1,7,2022
3,120.0,53.00,51.9,52.50,-5.25,-9.012876,8.0,412.0,2.398442,1.0,AAPL,C,1,6,2022,17,16,1,7,2022
4,125.0,57.11,46.9,47.55,0.00,0.000000,5.0,5.0,2.199223,1.0,AAPL,C,1,3,2022,20,31,1,7,2022


In [9]:
# Create Target
y = df_optionsML["lastPrice"]

# Create our features
# Convert string columns to numbers and drop 'lastPrice' column then assign to X
X = pd.get_dummies(df_optionsML, columns=['ticker', 'call_put',]).drop('lastPrice', axis=1)
X.head()

,strike,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,lastTradeDate_month,...,ticker_PG,ticker_TRV,ticker_UNH,ticker_V,ticker_VZ,ticker_WBA,ticker_WMT,ticker_ZVZZT,call_put_C,call_put_P
0,105.0,66.8,67.25,0.00,0.000000,6.0,12.0,2.468754,1.0,1,...,0,0,0,0,0,0,0,0,1,0
1,110.0,61.8,62.25,0.00,0.000000,8.0,4.0,2.250004,1.0,1,...,0,0,0,0,0,0,0,0,1,0
2,115.0,56.8,57.25,0.00,0.000000,3.0,3.0,2.031255,1.0,1,...,0,0,0,0,0,0,0,0,1,0
3,120.0,51.9,52.50,-5.25,-9.012876,8.0,412.0,2.398442,1.0,1,...,0,0,0,0,0,0,0,0,1,0
4,125.0,46.9,47.55,0.00,0.000000,5.0,5.0,2.199223,1.0,1,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Define the target set.
y = df_optionsML["lastPrice"].values
y[:5]

array([72.25, 68.45, 61.25, 53.  , 57.11])

In [11]:
# Splitting into Train and Test sets into an 80/20 split.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.80)

In [12]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(19100, 50)
(4776, 50)
(19100,)
(4776,)


In [13]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Creating the Linear Regression Model
model = LinearRegression()

In [15]:
model.fit(X_train, y_train)

LinearRegression()

In [16]:
y_pred = model.predict(X_test)
print(y_pred.shape)

(4776,)


In [17]:
from sklearn.metrics import accuracy_score
print(y_test, y_pred)

[2.620e+00 1.157e+02 9.000e-02 ... 1.000e-02 5.600e-01 3.050e+00] [8.63412096e-01 1.05515245e+02 8.69258228e-02 ... 2.44331391e-01
 1.44428399e+00 3.70623734e+00]


In [18]:
from sklearn import metrics
from sklearn.metrics import r2_score
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

print ('R^2 Coefficient of Determination:', r2_score(y_test, y_pred))


Mean Absolute Error: 3.7357772838330283
Mean Squared Error: 91.61968039304139
Root Mean Squared Error: 9.571816984932452
R^2 Coefficient of Determination: 0.9661659127061258


In [19]:
# Get complete array of prediction outputs
y_hats = model.predict(X)

# Create complete predictions dataframe
y_hats = pd.DataFrame(y_hats)

# Merge 'y_hats' dataframe on to 'df_optionsML' dataframe
df_optionsML['y_predict'] = y_hats[0]
df_optionsML

#----Delete below-----no .csv needed
# y_hats  = pd.DataFrame(y_hats)
# y_hats.to_csv('Resources/yhats.csv')

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,...,call_put,lastTradeDate_month,lastTradeDate_day,lastTradeDate_year,lastTradeDate_hour,lastTradeDate_min,exp_date_month,exp_date_day,exp_date_year,y_predict
0,105.0,72.25,66.80,67.25,0.00,0.000000,6.0,12.0,2.468754,1.0,...,C,1,5,2022,19,28,1,7,2022,65.334194
1,110.0,68.45,61.80,62.25,0.00,0.000000,8.0,4.0,2.250004,1.0,...,C,1,5,2022,14,35,1,7,2022,60.627798
2,115.0,61.25,56.80,57.25,0.00,0.000000,3.0,3.0,2.031255,1.0,...,C,1,5,2022,20,18,1,7,2022,56.374734
3,120.0,53.00,51.90,52.50,-5.25,-9.012876,8.0,412.0,2.398442,1.0,...,C,1,6,2022,17,16,1,7,2022,47.488187
4,125.0,57.11,46.90,47.55,0.00,0.000000,5.0,5.0,2.199223,1.0,...,C,1,3,2022,20,31,1,7,2022,47.045037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23871,165.0,30.90,0.00,0.00,0.00,0.000000,5.0,7.0,0.000010,1.0,...,P,11,29,2021,14,30,1,19,2024,5.609381
23872,170.0,33.70,35.80,37.80,0.00,0.000000,2.0,8.0,0.249519,1.0,...,P,10,20,2021,17,46,1,19,2024,40.813190
23873,185.0,46.80,46.55,52.35,0.00,0.000000,0.0,1.0,0.288551,1.0,...,P,11,10,2021,11,55,1,19,2024,50.402136
23874,195.0,61.50,52.85,56.25,0.00,0.000000,0.0,3.0,0.224373,1.0,...,P,12,13,2021,5,7,1,19,2024,54.203408
